In [61]:
import pandas as pd
import numpy as np

In [62]:
dados = pd.read_csv('Customer-Churn.csv') #LOAD DO ARQUIVO

In [63]:
dados.shape #FORMATO DO DATAFRAME(DF)

(7043, 18)

In [64]:
dados.head() #MOSTRA OS DADOS DO DF

,Maior65Anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,VariasLinhasTelefonicas,ServicoDeInternet,SegurancaOnline,BackupOnline,SeguroNoDispositivo,SuporteTecnico,TVaCabo,StreamingDeFilmes,TipoDeContrato,PagamentoOnline,FormaDePagamento,ContaMensal,Churn
0,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.85,Nao
1,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,ChequePapel,56.95,Nao
2,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,53.85,Sim
3,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30,Nao
4,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,70.70,Sim


In [65]:
traducao_dic = {'Sim': 1, 'Nao': 0}

dadosmodificados = dados[['Conjuge', 'Dependentes', 'TelefoneFixo', 'PagamentoOnline', 'Churn']].replace(traducao_dic) # para fazer a alteração no próprio dados, usar inplace=True

In [66]:
dummie_dados = pd.get_dummies(dados.drop(['Conjuge', 'Dependentes', 'TelefoneFixo', 'PagamentoOnline', 'Churn'], axis=1))

In [67]:
dados_finais = pd.concat([dadosmodificados, dummie_dados], axis=1)

In [68]:
# Para podermos aplicar o SMOTE, devemos separar  os dados em variáveis características e resposta  
X_antes = dados_finais.drop('Churn', axis = 1)
y_antes = dados_finais['Churn']

In [69]:
from imblearn.over_sampling import SMOTE

smt = SMOTE(random_state=123)  # Instancia um objeto da classe SMOTE
X_depois, y_depois = smt.fit_resample(X_antes,y_antes)  # Realiza a reamostragem do conjunto de dados

dados_finais = pd.concat([X_depois, y_depois], axis=1)  # Concatena a variável target (y) com as features (X)


In [70]:
X = dados_finais.drop('Churn', axis=1)
Y = dados_finais['Churn']

In [71]:
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()

X_normalizado = scaller.fit_transform(X)

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_normalizado, Y, test_size=0.3, random_state=123)

In [74]:
from sklearn.neighbors import KNeighborsClassifier

In [75]:
knn = KNeighborsClassifier(metric='euclidean')

In [76]:
knn.fit(X_treino, y_treino)

KNeighborsClassifier(metric='euclidean')

In [77]:
predito_knn = knn.predict(X_teste)

In [78]:
from sklearn.naive_bayes import BernoulliNB

In [79]:
bnb = BernoulliNB(binarize=np.abs(np.median(X_treino)))

In [80]:
bnb.fit(X_treino, y_treino)

predito_bnb = bnb.predict(X_teste)

In [81]:
from sklearn.tree import DecisionTreeClassifier

In [82]:
dtc = DecisionTreeClassifier(criterion='gini', random_state=42)

In [83]:
dtc.fit(X_treino, y_treino)

DecisionTreeClassifier(random_state=42)

In [84]:
predito_dtc = dtc.predict(X_teste)

In [85]:
from sklearn.metrics import confusion_matrix

In [87]:
print(confusion_matrix(y_teste, predito_knn))
print(confusion_matrix(y_teste, predito_bnb))
print(confusion_matrix(y_teste, predito_dtc))

[[1246  323]
 [ 262 1274]]
[[1060  509]
 [ 209 1327]]
[[1257  312]
 [ 288 1248]]


In [88]:
from sklearn.metrics import accuracy_score

In [89]:
print(accuracy_score(y_teste, predito_knn))
print(accuracy_score(y_teste, predito_bnb))
print(accuracy_score(y_teste, predito_dtc))

0.8115942028985508
0.7687600644122383
0.8067632850241546
